# Setup

In [2]:
import numpy as np
from scipy.special import comb
from scipy.optimize import linear_sum_assignment
from google.colab import drive
import time
from numba import njit,jit
drive.mount('/content/drive')
drive_path='drive/My Drive/Colab Notebooks/data_IISc experiment2/'

Mounted at /content/drive


In [3]:
!pip install thewalrus
import thewalrus as tw

     |████████████████████████████████| 6.6MB 2.4MB/s 
     |████████████████████████████████| 3.1MB 45.4MB/s 
     |████████████████████████████████| 5.8MB 43.2MB/s 
     |████████████████████████████████| 24.6MB 130kB/s 
  Found existing installation: llvmlite 0.31.0
    Uninstalling llvmlite-0.31.0:
      Successfully uninstalled llvmlite-0.31.0
  Found existing installation: numba 0.48.0
    Uninstalling numba-0.48.0:
      Successfully uninstalled numba-0.48.0
  Found existing installation: sympy 1.1.1
    Uninstalling sympy-1.1.1:
      Successfully uninstalled sympy-1.1.1


# Bethe Permanent

In [4]:
def permanent(A):
    return tw.perm(A)

In [ ]:
# np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})
np.set_printoptions(precision=6)
n=23
k=5/n
rng=np.random.default_rng() #937
P=random_sparse_matrix(rng,n,k)
perm=permanent(P)
while (np.isclose(perm,0)):
    P=random_sparse_matrix(rng,n,k)
    perm=permanent(P)
# P=rng.exponential(size=(n,n))
r,c=linear_sum_assignment(P,maximize=True)
P=P[:,c]
perm=permanent(P)
# print(P)
# P=np.reshape(np.array([range(n**2+1)],dtype=np.float64)[:,1:],(n,n))
# P=np.random.default_rng().uniform(size=(n,n))
# perm=permanent(P)
print(P)
m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=bp(P,100)
perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
ub=perm_bethe*np.sqrt(2)**P.shape[0]
eta=np.log2(perm/perm_bethe)/P.shape[0]
# eta_sum+=eta
# print(f'c: {c}, n: {n}, sample: {t}')
print("Perm: \t",perm)
print("Bethe: \t" ,perm_bethe)
print("Bounds: \t",perm_bethe<perm<ub)
print("Conv Value: ", conv_value)
print("Eta: \t",eta)
# print(belief_x)
print('\n##########')


In [ ]:
m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=belief_prop(P,100)
perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
ub=perm_bethe*np.sqrt(2)**P.shape[0]
eta=np.log2(perm/perm_bethe)/P.shape[0]
# eta_sum+=eta
# print(f'c: {c}, n: {n}, sample: {t}')
print("Perm: \t",perm)
print("Bethe: \t" ,perm_bethe)
print("Bounds: \t",perm_bethe<perm<ub)
print("Conv Value: ", conv_value)
print("Eta: \t",eta)

In [6]:
# @jit()
def bp(P,max_iter=50,eps=0.1):
  #initialize
    n=P.shape[0]
    phi=np.sqrt(P)
    psi=np.ones((n,n,n,n),dtype=np.bool)
    phi2=np.zeros((n,n,n,n),dtype=np.float128)
    prod=np.zeros(n)
    m_x2y=np.random.default_rng().random((n,n,n))
    m_y2x=np.random.default_rng().random((n,n,n))
    m_x2y_new=np.ones((n,n,n),dtype=np.float128) 
    m_y2x_new=np.ones((n,n,n),dtype=np.float128)

    belief_xy=np.zeros((n,n,n,n),dtype=np.float128)#belief_xy(i,j,xi,yj) is the value of b(x_i,y_j) for a given value of x_i and y_j
    belief_x=np.zeros((n,n),dtype=np.float128)
    belief_y=np.zeros((n,n),dtype=np.float128)

    # hash psi and phi(x_i)*phi(y_j)
    a=np.tile(np.array(range(n)),(n,1))
    for i in range(n):
      for j in range(n):
        psi[i,j,:,:]= np.logical_not((a==i)!=(a.T==j))
        phi2[i,j,:,:]=psi[i,j,:,:]*np.outer(phi[i,:],phi[:,j])

    for t in range(max_iter):

      #m_x2y    
      for i in range(n):
        for j in range(n):
          prod[:]=phi[i,:]
          for k in range(n):
            if k!=j:
              prod*=m_y2x[k,i,:]
          # print(np.sum(psi[i,j,:,:]*np.tile(prod[:,np.newaxis],(1,n)),axis=0))
          # print(prod)
          m_x2y_new[i,j,:]=np.sum(psi[i,j,:,:]*np.tile(prod[:,np.newaxis],(1,n)),axis=0)
      m_x2y_new=m_x2y_new/np.sum(m_x2y_new)
          # print(np.tile(prod,(1,n)))
          # print(psi[i,j,:,:]*np.tile(prod[:,np.newaxis],(1,n)))
          # print(np.sum(psi[i,j,:,:]*np.tile(prod[:,np.newaxis],(1,n)),axis=0))
          # print(np.repeat(prod[:,np.newaxis],(n,1)))
          # m_x2y_new[i,j,:]=np.sum(psi[i,j,:,:]*np.tile(prod[:,np.newaxis],(1,n)))
          # for yj in range(n):
          #   m_x2y_new[i,j,yj]=np.sum(psi[i,j,:,yj]*prod)
      # print(f'bp m {t} \n{m_x2y_new}')
      # print()

      #m_y2x
      for j in range(n):
        for i in range(n):
          prod[:]=phi[:,j]
          for k in range(n):
            if k!=i:
              prod*=m_x2y[k,j,:]
          m_y2x_new[j,i,:]=np.sum(psi[j,i,:,:]*np.tile(prod[:,np.newaxis],(1,n)),axis=0)
      m_y2x_new=m_y2x_new/np.sum(m_y2x_new)
      # print('bp m {t} \n',m_x2y)

      #update
      m_x2y_new.clip(min=0)
      m_y2x_new.clip(min=0)
#       print(np.log(m_x2y_new))
      m_x2y=(np.exp(np.nan_to_num(np.log(m_x2y) + eps*np.nan_to_num(np.log(m_x2y_new) - np.log(m_x2y)))))
      m_y2x=(np.exp(np.nan_to_num(np.log(m_y2x) + eps*np.nan_to_num(np.log(m_y2x_new) - np.log(m_y2x)))))
      m_x2y=m_x2y/np.sum(m_x2y) #normalization
      m_y2x=m_y2x/np.sum(m_y2x)#normalization
      #update using log rule
    #   for i in range(n):
    #     for j in range(n):
    #         for yj in range(n):
    #             if m_x2y_new[i,j,yj] >0:
    #                 m_x2y[i,j,yj]=np.exp(np.log(m_x2y[i,j,yj]) + eps*(np.log(m_x2y_new[i,j,yj]) - np.log(m_x2y[i,j,yj])))
    #             #   print("#",i,j,yj,(m_x2y[i,j,yj]),(m_x2y_new[i,j,yj]))
                
    #   for j in range(n):
    #     for i in range(n):
    #         for xi in range(n):
    #             if m_y2x_new[j,i,xi]>0:
    #                 m_y2x[j,i,xi]=np.exp(np.log(m_y2x[j,i,xi]) + eps*(np.log(m_y2x_new[j,i,xi]) - np.log(m_y2x[j,i,xi])))                   
                
    #   m_x2y=m_x2y/np.sum(m_x2y) #normalization
    #   m_y2x=m_y2x/np.sum(m_y2x)#normalization
        #update using log rule
#############################################################################
    #compute single node beliefs
    for i in range(n):
        for xi in range(n):
            prod_m_yk2xi=1
            for k in range(n):
                prod_m_yk2xi*=m_y2x[k,i,xi]
            belief_x[i,xi]=phi[i,xi]*prod_m_yk2xi
    # print(belief_x)
    
    for j in range(n):
        for yj in range(n):
            prod_m_xl2yj=1
            for l in range(n):
                prod_m_xl2yj*=m_x2y[l,j,yj]
            belief_y[j,yj]=phi[yj,j]*prod_m_xl2yj
            
    belief_xy=np.zeros((n,n,n,n),dtype=np.float128)
    for i in range(n):
        for j in range(n):
            for xi in range(n):
                for yj in range(n):
                    if not((xi==j)!=(yj==i)) and (m_x2y[i,j,yj]*m_y2x[j,i,xi])>0:
                        belief_xy[i,j,xi,yj]=belief_x[i,xi]*belief_y[j,yj]/(m_x2y[i,j,yj]*m_y2x[j,i,xi])
                    else:
                        belief_xy[i,j,xi,yj]=0
#     print(belief_xy[1,2])
    for i in range(n):
        for j in range(n):
            if np.sum(belief_xy[i,j,:,:])>0:
                belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])
#     print(belief_xy[1,2])
    
    #normalize single node beliefs
    for i in range(n):
        belief_x[i,:]=belief_x[i,:]/np.sum(belief_x[i,:])
            
    for j in range(n):
        belief_y[j,:]=belief_y[j,:]/np.sum(belief_y[j,:])
        
#     belief_xy=belief_xy/n
#     belief_x=belief_x/(n)
#     belief_y=belief_y/(n)
        
    return m_x2y,m_y2x,belief_xy,belief_x,belief_y,(np.sum(np.abs(m_y2x-m_y2x_new))<1e-10),np.sum(np.abs(m_y2x-m_y2x_new))

In [7]:
def bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x):
        
    t1=t2=t3=t4=t5=t6=0
    n=P.shape[0]
    phi=np.sqrt(P)
    
    for i in range(n):
        for j in range(n):
            for xi in range(n):
                for yj in range(n):
                    if belief_xy[i,j,xi,yj]!=0 and not((xi==j)!=(yj==i)):
                        t1+=belief_xy[i,j,xi,yj]*np.log(phi[i,xi]*phi[yj,j])
                        t2+=belief_xy[i,j,xi,yj]*np.log(belief_xy[i,j,xi,yj])
    
    for i in range(n):
        for xi in range(n):
            if belief_x[i,xi]!=0:
                t3+=belief_x[i,xi]*np.log(belief_x[i,xi])
                t5+=belief_x[i,xi]*np.log(phi[i,xi])
                                          
    for j in range(n):
        for yj in range(n):
            if belief_y[j,yj]!=0:
                t4+=belief_y[j,yj]*np.log(belief_y[j,yj])
                t6+=belief_y[j,yj]*np.log(phi[yj,j])
    
    energy=-t1+t2-(n-1)*t3-(n-1)*t4+(n-1)*t5+(n-1)*t6
    return np.exp(-energy)

# Experiments

#### Function to generate random matrices for given $\kappa$

In [8]:
def random_sparse_matrix(rng,n,kappa):
    mask=rng.binomial(1,kappa,(n,n))
    mat=rng.exponential(1,size=(n,n))
    # mat=rng.uniform(high=2.0,size=(n,n)) #mean=1
    # mat=rng.gamma(1,size=(n,n))
    return np.multiply(mask,mat)


#### Test 

#### Compute permanent for each n with sparsity c/n

In [ ]:
rng=np.random.default_rng(867)
np.seterr(divide='warn', invalid='warn')
for c in np.array([5.0]): #np.linspace(1,3,num=9)[0:1]:
    eta_avg_arr=np.zeros(30)
    samples=100
    # print('c: ',c)
    for n in range(23,25):
        eta_sum=0
        k=c/n
        # print('n: ',n)
        for t in range(0,samples):
            P=random_sparse_matrix(rng,n,k)
            perm=permanent(P)
            
            while (np.isclose(perm,0)):
                P=random_sparse_matrix(rng,n,k)
                perm=permanent(P)
            print("Perm: \t",perm)
            #preprocessing step, vatedka et al
            r,c1=linear_sum_assignment(P,maximize=True)
            P=P[:,c1]
            
            # print(P)
            m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=bp(P,80,eps=0.1)
            perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
            ub=perm_bethe*np.sqrt(2)**P.shape[0]
            eta=np.log2(perm/perm_bethe)/P.shape[0]
            eta_sum+=eta
            print(f'c: {c}, n: {n}, sample: {t}')
            print("Bethe: \t" ,perm_bethe)
            print("Bounds: \t",perm_bethe<perm<ub)
            print("Conv Value: ", conv_value)
            print("Eta: \t",eta)
            print('\n\n\n')
            # np.savez(drive_path+f'linear_exp/exp_linear_c_{c}_n_{n}_sample_{t}',
            #          perm=perm,perm_bethe=perm_bethe,eta=eta)
        eta_avg_arr[n]=eta_sum/samples
        print('eta_avg_arr \n\n\n',eta_avg_arr)
    np.save(drive_path+f'linear_exp/eta_exp_linear_c_{c}_bp_n_22_24',eta_avg_arr)

Perm: 	 1.567009990845228


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in subtract
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in subtract


c: 5.0, n: 23, sample: 0
Bethe: 	 0.33603366956199296744
Bounds: 	 True
Conv Value:  1.5890204544953132948
Eta: 	 0.09657985560986237893




Perm: 	 294.56479288336135
c: 5.0, n: 23, sample: 1
Bethe: 	 79.344514944083514185
Bounds: 	 True
Conv Value:  1.725092007756661817
Eta: 	 0.08227750467450990065




Perm: 	 153993.80138644052
c: 5.0, n: 23, sample: 2
Bethe: 	 26395.676397863581107
Bounds: 	 True
Conv Value:  1.1740507771739827643
Eta: 	 0.11063037996016730904




Perm: 	 1207.8188035855153
c: 5.0, n: 23, sample: 3
Bethe: 	 293.38001046317270618
Bounds: 	 True
Conv Value:  1.5350138620823212317
Eta: 	 0.0887635459762334037




Perm: 	 728.9270510983765
c: 5.0, n: 23, sample: 4
Bethe: 	 176.75520115383335763
Bounds: 	 True
Conv Value:  1.4762458639659409283
Eta: 	 0.0888705118699704618




Perm: 	 1600857.1222199197
c: 5.0, n: 23, sample: 5
Bethe: 	 264720.41675980258904
Bounds: 	 True
Conv Value:  1.3981272890130453807
Eta: 	 0.11288274690186645626




Perm: 	 2478619.0612988006
c

#### Compute permanent for each $\kappa$ and n


In [ ]:
rng=np.random.default_rng(2)
for n in range(3,10):
    samples=10
    eta_avg_arr=np.zeros(10)
    idx=0
    for k in np.linspace(0,1,num=11)[1:10]:
        eta_sum=0
        print('k: ',k)
        for t in range(0,samples):
            P=random_sparse_matrix(rng,n,k)
            perm=permanent(P)
            
            while (np.isclose(perm,0)):
                P=random_sparse_matrix(rng,n,k)
                perm=permanent(P)

            # print(P)
            m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=belief_prop(P,300)
            perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
            ub=perm_bethe*np.sqrt(2)**P.shape[0]
            eta_sum+=np.log(perm/perm_bethe)/P.shape[0]

            print("Perm: \t",perm)
            print("Bethe: \t" ,perm_bethe)
            print("Bounds: \t",perm_bethe<perm<ub)
            print("Convergence: \t",conv)
            if not conv:
                print("Conv Value: ", conv_value)
            print("Eta: \t",np.log(perm/perm_bethe)/P.shape[0])
            print('\n\n\n')
        eta_avg_arr[idx]=eta_sum/samples
        idx+=1
    np.save('eta_avg_arr_gamma_n_'+str(n),eta_avg_arr)

#### Print table

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})
path=drive_path+'eta_exp_linear_c_'
data1=np.load(path+'1.0.npy')
data2=np.load(path+'1.25.npy')
data3=np.load(path+'1.5.npy')
data4=np.load(path+'1.75.npy')
data5=np.load(path+'2.0.npy')
print(data1[3:]*np.log2(np.e))
print(data2[3:]*np.log2(np.e))
print(data3[3:]*np.log2(np.e))
print(data4[3:]*np.log2(np.e))
print(data5[3:]*np.log2(np.e))

# path=drive_path+'gamma/eta_avg_arr_gamma_n_'
# data1=np.load(path+'3.npy')
# data2=np.load(path+'4.npy')
# data3=np.load(path+'5.npy')
# data4=np.load(path+'6.npy')
# data5=np.load(path+'7.npy')
# data6=np.load(path+'8.npy')
# data7=np.load(path+'9.npy')

# print(data1*np.log2(np.e))
# print(data2*np.log2(np.e))
# print(data3*np.log2(np.e))
# print(data4*np.log2(np.e))
# print(data5*np.log2(np.e))
# print(data6*np.log2(np.e))
# print(data7*np.log2(np.e))

In [13]:
file1=drive_path+'linear_exp/eta_exp_linear_c_1.0_bp.npy'
# file2=drive_path+'linear_exp/eta_exp_linear_c_2.0.npy'
# file3=drive_path+'linear_exp/eta_exp_linear_c_2.0_n_9_to_11.npy'
# file4=drive_path+'linear_exp/eta_exp_linear_c_3.0.npy'
# file5=drive_path+'linear_exp/eta_exp_linear_c_4.0_bp.npy'
# file6=drive_path+'linear_exp/eta_exp_linear_c_5.0_bp_n_22_24.npy'
# file=drive_path+'linear_uniform/eta_uniform_linear_c_1.0.npy'
# file=drive_path+'linear_uniform/eta_uniform_linear_c_2.0.npy'
# file=drive_path+'linear_uniform/eta_uniform_linear_c_3.0.npy'
# file=drive_path+'linear_gamma/eta_gamma_linear_c_1.0.npy'
# file=drive_path+'linear_gamma/eta_gamma_linear_c_2.0.npy'
# file=drive_path+'linear_gamma/eta_gamma_linear_c_3.0.npy'
# np.save(file1,np.concatenate([np.load(file1)[0:12],np.load(file2)[12:16],np.load(file3)[16:]]))
# c5.0 n22 0.106848
print(np.load(file1))
# print(np.load(file6))

[0.         0.         0.         0.04270496 0.03337238 0.02283119
 0.03281484 0.01821778 0.02403862 0.02201298 0.0202531  0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]


# Scratch

In [ ]:
p=np.ones((15,15))
s=time.time()
# perm=bethe(p)
m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=belief_prop(p,300)
perm_bethe=bfe(p,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
e=time.time()
print(e-s)
#308.0300898551941 sec for 20 x 20
#137.48850893974304 sec for 19 x 19
#5.661278247833252 sec for 15

#bp
#244.77203011512756 for 15

244.77203011512756


In [14]:
%cd drive/'My Drive'/'Colab Notebooks'/'data_IISc experiment2'
!pwd

/content/drive/My Drive/Colab Notebooks/data_IISc experiment2
/content/drive/My Drive/Colab Notebooks/data_IISc experiment2


In [16]:
!pwd
!cp ./linear_exp/eta* .

/content/drive/My Drive/Colab Notebooks/data_IISc experiment2
